In [1]:
import numpy as np
import pulp
from pulp.apis import PULP_CBC_CMD

## Jobshop solver using PULP
Each job has a start time, end time and reward (not cost here).
Objective is to maximize reward by choosing which jobs to do such that there is no
overlap and they complete in the specified time. Note: in this formulation start times
are fixed, i.e., each job has to start at the stipulated time (which can therefore default
to a large value, e.g. 75).

In [13]:
class Jobs():
    def __init__(self,T=75):
        self.rL=[]
        self.eL=[]
        self.tL=[]
        self.T=T
        self.k=0
        self.dL=[] #descriptions for use outside jobshop
    def add_job(self,t,e,r,d=[]):
        self.rL+=[r]
        self.eL+=[e]
        self.tL+=[t]
        self.dL+=[d]
        self.k=len(self.rL)
        self.x=np.zeros(self.k)
    def setup(self):
        x=[]
        for k in range(self.k):
            x+=[pulp.LpVariable('x'+str(k), lowBound=0, cat='Integer')]
        self.objective = pulp.LpProblem('Optimal Schedule', pulp.LpMaximize)
        expr=0
        tot_time=0
        consL=[]
        T=5
        for i in range(self.k):
            expr+=x[i]*self.rL[i]
            tot_time+=x[i]*(self.eL[i]-self.tL[i])
            for j in range(self.k):
                if self.tL[j]>=self.tL[i] and j is not i:
                    consL+=[x[i]*self.eL[i]<=x[j]*self.tL[j]+(1-x[j])*self.T]
                if self.tL[j]<=self.tL[i] and j is not i:
                    consL+=[x[j]*self.eL[j]<=x[i]*self.tL[i]+(1-x[i])*self.T]
        self.objective+=expr
        self.objective+=tot_time<=self.T
        for c in consL:
            self.objective+=c
        for i in range(self.k):
            self.objective+=x[i]<=1
    def solve(self,verbose=False):
        self.objective.solve(PULP_CBC_CMD(msg=verbose))
    def get_soln(self,verbose=False):
        if verbose: print('Optimal Solution: ')
        for variable in self.objective.variables():
            if verbose:print(variable.name, '=', variable.varValue)
            self.x[int(variable.name[1:])]=variable.varValue
        return self.x

In [14]:
# # pulp.listSolvers(onlyAvailable=True)
# # Example with 5 jobs
# jobs=Jobs()
# jobs.add_job(0,3,1)
# jobs.add_job(0,2,-1)
# jobs.add_job(1,4,5)
# jobs.add_job(3,5,3)
# jobs.add_job(2,4,1)
# jobs.setup()
# jobs.solve()
# jobs.get_soln()

array([0., 0., 1., 0., 0.])

In [136]:
# # import PuLP
# import pulp

# # define decision variables
# x1 = pulp.LpVariable('x1', lowBound=0, cat='Integer')
# x2 = pulp.LpVariable('x2', lowBound=0, cat='Integer')

# # define objective function
# objective = pulp.LpProblem('Maximize Profit', pulp.LpMaximize)
# objective += 10 * x1 + 20 * x2

# # define constraints
# constraint1 = 10 * x1 + 15 * x2 <= 150
# constraint2 = x1 + x2 <= 20

# # add constraints to the objective
# objective += constraint1
# objective += constraint2

# # solve the problem
# objective.solve()

# # print results
# print('Optimal Solution: ')
# for variable in objective.variables():
#     print(variable.name, '=', variable.varValue)

# print('Optimal Profit = ', pulp.value(objective.objective))